# RAG with Vector Search

### Download and Flatten FAQ Documents
This cell downloads the FAQ documents from a remote JSON file, then flattens the nested structure so each document includes its course name. This makes it easier to process and search the documents later.

In [1]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

### Create and Fit Minsearch Index
This cell imports Minsearch, creates an index specifying which fields to use for text and keyword search, and fits (trains) the index on the loaded documents. This prepares the index for efficient search queries.

In [3]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
from openai import OpenAI

openai_client = OpenAI()

### Define Search Function with Boosting and Filtering
This cell defines a function to search the Minsearch index. It applies field boosting (giving more weight to the 'question' field) and filters results by course, returning the top 5 most relevant documents.

In [5]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

### Build Prompt for LLM
This cell defines a function to build a prompt for the language model. It combines the user query and the retrieved FAQ context into a single prompt, which will be sent to the LLM for answer generation.

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

### Define LLM Call Function
This cell defines a function that sends a prompt to the OpenAI chat model and returns the generated answer. The prompt typically includes the user query and relevant FAQ context.

In [7]:
def llm(prompt):
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini', # R: Cheaper
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

### Define RAG Pipeline Function
This cell defines a function that implements the full Retrieval-Augmented Generation (RAG) pipeline: it searches the Minsearch index, builds a prompt from the results, and calls the LLM to generate an answer.

In [ ]:
def rag(query):
    search_results = search(query) # R: Search the index for the query 
    prompt = build_prompt(query, search_results) # R: Build the prompt using the search results
    answer = llm(prompt) # R: Call the LLM with the prompt (provide llm with context)
    return answer

In [9]:
rag('how do I run kafka?') #R: Call the RAG function with a query

'To run Kafka using Java, navigate to your project directory and execute the following command in the terminal:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you are using Python, ensure that you create a virtual environment, install the necessary packages from `requirements.txt`, and then run your Python files within that environment. Here are the steps:\n\n1. Create a virtual environment and install packages:\n   ```\n   python -m venv env\n   source env/bin/activate  # For MacOS/Linux\n   pip install -r ../requirements.txt\n   ```\n\n2. Activate the virtual environment (you will need to do this each time):\n   ```\n   source env/bin/activate  # For MacOS/Linux\n   ```\n\n   On Windows, use:\n   ```\n   env\\Scripts\\activate\n   ```\n\n3. To deactivate the virtual environment when you are done, run:\n   ```\n   deactivate\n   ```\n\nMake sure Docker images are up and running if you are using them in your setup.'

In [10]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course after it has started. Even if you don't register, you are eligible to submit homework. However, keep in mind that there will be deadlines for the final projects, so it's advisable not to leave everything until the last minute."

### Set Up Qdrant Vector Database and Client
This cell (and the following ones) set up Qdrant, a vector database for similarity search. It installs the required package, imports the client, and connects to a local Qdrant server.

In [11]:
# !python -m pip install -q "qdrant-client[fastembed]>=1.14.2" # Install quadrant into environment
from qdrant_client import QdrantClient, models # Qdrant as python module

In [12]:
qd_client = QdrantClient("http://localhost:6333") # Not client instance as above, but server connection

### Configure Qdrant Collection and Embedding Model
This cell sets up the configuration for the Qdrant collection, including the embedding dimensionality and the model used to generate vector embeddings for the documents.

In [13]:
# Set-up the configuration
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [14]:
collection_name = "zoomcamp-faq"

### Delete Existing Qdrant Collection (if any)
This cell deletes any existing Qdrant collection with the same name to ensure a clean setup before creating a new collection for the FAQ documents.

In [27]:
qd_client.delete_collection(collection_name=collection_name)

True

### Create Qdrant Collection for Vector Search
This cell creates a new Qdrant collection with the specified vector size and distance metric (cosine similarity). This collection will store the vector embeddings for the FAQ documents.

In [28]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

### Create Payload Index for Course Filtering
This cell creates a payload index in Qdrant for the 'course' field, enabling efficient filtering of search results by course name.

In [29]:
qd_client.create_payload_index( 
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

### Create Vector Points for Qdrant
This cell creates vector points for each FAQ document by combining the question and answer text, generating embeddings, and packaging them with their metadata. These points will be inserted into the Qdrant collection.

In [30]:
points = []

for i, doc in enumerate(documents): # Documents already unnested above
    text = doc['question'] + ' ' + doc['text']# Combine Q&A in the index
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)# Text, Model, Payload

In [31]:
points[0]

PointStruct(id=0, vector=Document(text="Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with ann

### Upsert Vector Points into Qdrant Collection
This cell inserts (or updates) all the vector points into the Qdrant collection, making the FAQ documents available for vector similarity search.

In [32]:
# Upsert ("update" and "insert") DB
qd_client.upsert( # May be slow
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [33]:
question = 'I just discovered the course. Can I still join it?'

### Define Vector Search Function for Qdrant
This cell defines a function to perform vector similarity search in Qdrant. It embeds the user query, filters by course, and retrieves the top 5 most similar FAQ documents.

In [34]:
def vector_search(question):
    print('vector_search is used')

    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name, # Perform search in the collection
        query=models.Document(
            text=question, # Embedde the query in the document
            model=model_handle
        ),
        query_filter=models.Filter( # Filter the courses
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )

    results = []

    for point in query_points.points:
        results.append(point.payload)# Same as query_points.points[0].payload

    return results

### Define RAG Pipeline with Qdrant Vector Search
This cell defines a RAG pipeline function that uses Qdrant for vector search, builds a prompt from the retrieved documents, and calls the LLM to generate an answer.

In [35]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
rag('how do I run kafka?')

vector_search is used


"To run Kafka, you need to follow a few steps depending on what you are using (producer, consumer, etc.). Here are the general instructions:\n\n1. **Run the Producer/Consumer in the terminal**: \n   Navigate to the project directory and run the following command:\n   ```\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n   Replace `<jar_name>` with the actual name of your JAR file.\n\n2. **Ensure Kafka Broker is Running**: \n   If you encounter the error `kafka.errors.NoBrokersAvailable`, check if your Kafka broker Docker container is running. You can confirm this by running `docker ps` in the terminal. If it's not running, start it by navigating to the directory containing the Docker Compose YAML file and executing:\n   ```\n   docker compose up -d\n   ```\n\n3. **Check Configuration**: \n   Make sure that the `StreamsConfig.BOOTSTRAP_SERVERS_CONFIG` in your scripts is set to the correct server URL. Additionally, verify that y

: 